In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from IPython.display import display, Markdown, Image

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8), 'figure.dpi': 150,
                     'axes.titlesize': 'large', 'axes.labelsize': 'medium',
                     'xtick.labelsize': 'small', 'ytick.labelsize': 'small'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for Vector Autoregression (VAR) analysis.")

# Part 6: Econometrics
## Chapter 6.8: Vector Autoregression (VAR) Models

### Introduction: From Univariate to Multivariate Time Series

While classical time series models like ARMA are powerful for understanding the dynamics of a single variable, economic phenomena rarely occur in isolation. Interest rates, inflation, and unemployment all interact and influence each other. **Vector Autoregression (VAR)**, introduced by Christopher Sims (1980), provides a framework for analyzing these rich, dynamic interdependencies in multivariate time series data.

The genius of the VAR approach lies in its simplicity and minimal theoretical restrictions. Instead of imposing a structural model based on economic theory (which might be misspecified), a VAR treats every variable in the system as potentially endogenous. Each variable is modeled as a linear function of its own past values and the past values of all other variables in the system. This atheoretical starting point allows the data to speak for itself.

This notebook will:
1.  Outline the mathematical structure of a VAR(p) model.
2.  Discuss the estimation of VAR models using Ordinary Least Squares (OLS).
3.  Introduce the crucial concept of **structural identification** and explain how **Cholesky decomposition** is used to recover orthogonal shocks.
4.  Demonstrate how to estimate a VAR and generate **Impulse Response Functions (IRFs)** to analyze the dynamic effects of macroeconomic shocks using the `statsmodels` library.

### 1. The VAR(p) Model

A VAR(p) model expresses a vector of $k$ endogenous variables, $y_t$, as a linear function of $p$ of its own lags and a vector of innovations (shocks), $u_t$.

For a simple VAR(1) with two variables ($y_{1,t}$ and $y_{2,t}$), the system is:
$$ y_{1,t} = c_1 + \phi_{11,1} y_{1,t-1} + \phi_{12,1} y_{2,t-1} + u_{1,t} $$ 
$$ y_{2,t} = c_2 + \phi_{21,1} y_{1,t-1} + \phi_{22,1} y_{2,t-1} + u_{2,t} $$

In matrix form, a general VAR(p) is written as:
$$ y_t = c + \Phi_1 y_{t-1} + \Phi_2 y_{t-2} + ... + \Phi_p y_{t-p} + u_t $$

where $y_t$ is a ($k \times 1$) vector, $c$ is a ($k \times 1$) vector of intercepts, $\Phi_i$ are ($k \times k$) coefficient matrices, and $u_t$ is a ($k \times 1$) vector of innovations with covariance matrix $E[u_t u_t'] = \Sigma$. The key assumption is that the innovations are serially uncorrelated but may be contemporaneously correlated (i.e., $\Sigma$ is not necessarily diagonal).

### 2. Estimation

Because each equation in the VAR system has the same set of regressors (the lagged values of all variables), the system can be estimated efficiently and consistently by applying Ordinary Least Squares (OLS) to each equation individually. This simplifies the estimation process significantly.

### 3. Structural VARs and Identification

The estimated innovations, $u_t$, are called **reduced-form shocks**. They are the one-step-ahead forecast errors from the model, but they are not economically meaningful as "structural" shocks (e.g., a pure monetary policy shock or technology shock). This is because they are contemporaneously correlated ($\Sigma$ is not diagonal). For example, a surprise increase in the federal funds rate ($u_{FFR,t}$) might be correlated with a simultaneous change in expected inflation ($u_{INF,t}$), so it isn't a clean policy shock.

To identify structural shocks, $\epsilon_t$, which are by definition orthogonal, we need to impose restrictions. The relationship between the reduced-form and structural shocks is:
$$ A u_t = B \epsilon_t \implies u_t = A^{-1} B \epsilon_t $$  
The goal is to find the matrices $A$ and $B$. A common and simple identification scheme is the **Cholesky decomposition**. It imposes a recursive ordering on the variables. The first variable is assumed to be affected only by its own structural shock contemporaneously. The second variable is affected by its own shock and the first shock, and so on. This corresponds to choosing $B=I$ and making $A$ a lower triangular matrix. This is achieved by finding the Cholesky factor of the reduced-form covariance matrix $\Sigma$.

### 4. Code Example: A Monetary Policy VAR

We will estimate a simple VAR for the U.S. economy using quarterly data on GDP growth, inflation, and the federal funds rate. We will then trace out the effects of a monetary policy shock (an unexpected increase in the federal funds rate) using IRFs.

In [ ]:
sec("Estimating a Monetary Policy VAR")

# Load classic macro data from statsmodels
data = sm.datasets.macrodata.load_pandas().data
data['year'] = data['year'].astype(int)
data.index = pd.to_datetime(data['year'].astype(str) + 'Q' + data['quarter'].astype(str))

# Prepare the data: GDP growth, inflation (from CPI), and the fed funds rate
df = pd.DataFrame({
    'gdp_growth': 100 * data['realgdp'].pct_change(),
    'inflation': 100 * data['cpi'].pct_change(),
    'fed_funds': data['tbilrate'] # Using t-bill rate as a proxy
}).dropna()

# --- Estimate the VAR model ---
model = VAR(df)
results = model.fit(maxlags=15, ic='aic') # Use AIC to select the optimal lag order
note(f"Optimal lag order chosen by AIC: {results.k_ar}")

# --- Generate and Plot Impulse Responses ---
note("Impulse responses to a 1 S.D. shock to the Federal Funds Rate:")
irf = results.irf(periods=20)
fig = irf.plot(orth=True, signif=0.05) # Orth=True applies the Cholesky decomposition
fig.suptitle('Impulse Responses to a Monetary Policy Shock (Cholesky)', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

note("**Interpretation:** The results show a classic monetary policy trade-off. An unexpected 1 S.D. increase in the federal funds rate (a contractionary shock) leads to a statistically significant fall in inflation and a temporary, significant decline in GDP growth. This illustrates the power of VARs to uncover dynamic relationships in the data without imposing strong theoretical assumptions.")

### 5. Exercises

1.  **Alternative Ordering:** The Cholesky decomposition is sensitive to the ordering of the variables. Re-estimate the VAR, but change the order of the variables in the DataFrame to `['inflation', 'gdp_growth', 'fed_funds']`. How do the impulse responses to a federal funds rate shock change? Why? This highlights the importance of the identifying assumptions.

2.  **Forecast Error Variance Decomposition (FEVD):** The FEVD shows the proportion of the forecast error variance for each variable that is attributable to its own shocks versus shocks to other variables. Use the `results.fevd()` method to compute and plot the FEVD for a 20-quarter horizon. What does it tell you about the relative importance of monetary policy shocks for explaining fluctuations in GDP growth and inflation?

3.  **Historical Decomposition:** The `results.plot_acorr()` method can be used to plot the historical decomposition of the variables, showing how the different structural shocks have contributed to their evolution over time. Generate and interpret this plot.

4.  **Granger Causality:** Use the `results.test_causality()` method to test whether the federal funds rate "Granger-causes" GDP growth. What is the null hypothesis of this test, and what does the result imply?